In [1]:
def query_distinct_data(athena_client,result_output_location):
    query = "SELECT DISTINCT * from fire_data.rfs_fire_data where state in ('NSW','VIC','QLD')"
    response = athena_client.start_query_execution(
        QueryString=query,
        ResultConfiguration={"OutputLocation": result_output_location})
    return response["QueryExecutionId"]
    
    


In [2]:
def distinct_data_df(pd,s3_client,bucket, response_path):
    #bucket = "camgoo2-rfs-visualisation"
    #response_path = "query_results/"+execution_id+'.csv'

     
    # 's3' is a key word. create connection to S3 using default config and all buckets within S3
    obj = s3_client.get_object(Bucket= bucket, Key= response_path) 
    # get object and file (key) from bucket
    df = pd.read_csv(obj['Body']) # 'Body' is a key word
    df= df.dropna()
    return df


In [3]:
def create_distinct_data_map(df,folium):
    fire_location = df[["latitude", "longitude"]]
    map = folium.Map(location=[fire_location.latitude.mean(), fire_location.longitude.mean()],
                 zoom_start=5, control_scale=True)
    for index, location_info in fire_location.iterrows():
        folium.Marker([location_info["latitude"], location_info["longitude"]]
                      ,icon=folium.Icon(color="orange",icon = "fire")).add_to(map)
    #have used https://mapshaper.org/ to reduce the file size of polygons. I am now using 
    #reduced file LGA11aAust.json rather than LGA11aAust.geojson 
    folium.GeoJson('LGA11aAust.json').add_to(map)
    
    return map